In [1]:
import datacube
from datacube.storage.masking import mask_invalid_data
from datacube.api.query import Query
dc = datacube.Datacube()

In [2]:
dc.list_measurements()

aliases  dtype  \
product    measurement                                         
LC08_scene coastal_aerosol  [band_1, coastal_aerosol]  int16   
           blue                        [band_2, blue]  int16   
           green                      [band_3, green]  int16   
           red                          [band_4, red]  int16   
           nir                          [band_5, nir]  int16   
           swir1                      [band_6, swir1]  int16   
           swir2                      [band_7, swir2]  int16   
           panchromatic        [band_8, panchromatic]  int16   
           cirrus                    [band_9, cirrus]  int16   
           lwir1                     [band_10, lwir1]  int16   
           lwir2                     [band_11, lwir2]  int16   
           quality                 [QUALITY, quality]  int16   
LE07_scene blue                        [band_1, blue]  int16   
           green                      [band_2, green]  int16   
           red                          [band_3, red]  int16   
           nir                          [band_4, nir]  int16   
           swir1                      [band_5, swir1]  int16   
           swir2                      [band_7, swir2]  int16   
           quality                 [QUALITY, quality]  int16   
LM03_scene green                      [band_4, green]  int16   
           red                          [band_5, red]  int16   
           nir1                        [band_6, nir1]  int16   
           nir2                        [band_7, nir2]  int16   
           quality                 [QUALITY, quality]  int16   
LM05_scene green                      [band_1, green]  int16   
           red                          [band_2, red]  int16   
           nir1                        [band_3, nir1]  int16   
           nir2                        [band_4, nir2]  int16   
           quality                 [QUALITY, quality]  int16   
LT05_scene blue                        [band_1, blue]  int16   
           green                      [band_2, green]  int16   
           nir                          [band_4, red]  int16   
           red                          [band_3, nir]  int16   
           swir1                      [band_5, swir1]  int16   
           swir2                      [band_7, swir2]  int16   
           quality                 [QUALITY, quality]  int16   

                                                             flags_definition  \
product    measurement                                                          
LC08_scene coastal_aerosol                                                NaN   
           blue                                                           NaN   
           green                                                          NaN   
           red                                                            NaN   
           nir                                                            NaN   
           swir1                                                          NaN   
           swir2                                                          NaN   
           panchromatic                                                   NaN   
           cirrus                                                         NaN   
           lwir1                                                          NaN   
           lwir2                                                          NaN   
           quality          {'cloud': {'bits': [4], 'values': {'0': False,...   
LE07_scene blue                                                           NaN   
           green                                                          NaN   
           red                                                            NaN   
           nir                                                            NaN   
           swir1                                                          NaN   
           swir2                                                          

In [7]:
# Declare some 'constants' to use in program

fake_saturation = 10000
p = 'LM05_scene'
res = (0.0025, -0.0025)
crs = 'EPSG:4236'
m1 = 'red'
m2 = 'green'
m3 = 'nir1'
start = '1978'
end = '1980'

#query = Query(product=p, time=(start, end), )

In [8]:
dc.find_datasets(product = p)

[Dataset <id=b2e8bbbe-4095-538f-9e69-e946dc453e2e type=LM05_scene location=/data/datacube/in/landsat/LM05_L1TP_016020_20120831_20180521_01_T1/metadata.yaml>,
 Dataset <id=ef2f98f6-2f49-5be0-ab2c-16710e7c62f6 type=LM05_scene location=/data/datacube/in/landsat/LM05_L1TP_017028_20120806_20180521_01_T2/metadata.yaml>]

In [ ]:
data = dc.load( product = p
               , measurements = [m1, m2, m3]
               , time = (start, end)
               #, **query.search_terms
              )

In [ ]:
# Use the masking library to strip invalid data
data = mask_invalid_data(data)
# Isolate the color dimension in an xarray.DataArray, use transpose to make color the last dimension
rgb = (data.to_array(dim='color'))
rgb = rgb.transpose(*(rgb.dims[1:]+rgb.dims[:1])) 

# Mask out pixels where any band is 'saturated'
rgb = rgb.where((rgb <= fake_saturation).all(dim='color'))
# Scale to [0, 1] range for imshow
rgb /= fake_saturation  

In [ ]:
rgb.plot.imshow( x = data.crs.dimensions[1]
                , y = data.crs.dimensions[0]
                , col = 'time'
                , col_wrap = 1
                , size = 10
                , add_colorbar = False
               )

In [ ]:
for d in data.data_vars:
    (data.data_vars[d].loc[ '2000' : '2012' ]).plot(col = 'time', col_wrap = 10)